In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
sys.path.append('../utils')

In [3]:
# data=pd.read_csv('../data/expanded_crsp_stock_data_modern.csv')
data=pd.read_csv('../data/NVDA_30min_sample.csv')
data.head(5)

,timestamp,open,high,low,close,volume
0,2024-02-26 04:00:00,790.90,798.00,787.87,797.98,27109
1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848
2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654
3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711
4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169


In [40]:
# Assuming 'data' is your original stock data and 'factors' is your factor data
# Convert 'date' in 'data' DataFrame to datetime
data['date'] = pd.to_datetime(data['date'])
data['MonthYear'] = data['date'].dt.to_period('M')

# Convert non-numeric 'RET' to NaN and then drop those rows
data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
data = data.dropna(subset=['RET'])

# # Convert 'date' in 'factors' DataFrame from 'YYYYMM' format to datetime
# factors['date'] = pd.to_datetime(factors['date'], format='%Y%m')
# factors['MonthYear'] = factors['date'].dt.to_period('M')

# # Forward fill the factors for each month
# factors = factors.set_index('date').resample('D').ffill().reset_index()
# factors['MonthYear'] = factors['date'].dt.to_period('M')

# Calculate Market Cap
data['MktCap'] = data.apply(lambda x: abs(x['PRC']) * x['SHROUT'] if x['PRC'] != 0 else pd.NA, axis=1)
data['MktCap'] = pd.to_numeric(data['MktCap'], errors='coerce')  # Convert Market Cap to numeric, handling errors

# Drop rows where Market Cap is NA
data = data.dropna(subset=['MktCap'])

# Selecting the last day of each month and the top 500 companies based on Market Cap
last_day_each_month = data.groupby('MonthYear')['date'].transform('max')
data_last_day = data[data['date'] == last_day_each_month]

# Select the Top 500 firms based on Market Cap on the last day of each month
top500_last_day = data_last_day.groupby('MonthYear').apply(lambda x: x.nlargest(500, 'MktCap')).reset_index(drop=True)

# Preparing for Next Month's Data
top500_last_day['NextMonth'] = top500_last_day['MonthYear'].apply(lambda x: x + 1)
top500_map = top500_last_day[['NextMonth', 'TICKER']].drop_duplicates()

# Merging with the original data to create the stock universe for the next month
data['NextMonth'] = data['MonthYear']
resulting_df = pd.merge(data, top500_map, on=['NextMonth', 'TICKER'])

# Merge the factors with the filtered stock data
# resulting_df = pd.merge(resulting_df, factors, on='date', how='left')

# Select Relevant Columns for the Resulting DataFrame
# resulting_df = resulting_df[['date', 'TICKER', 'RET', 'MktCap', 'Mkt-RF', 'SMB', 'HML', 'RF']]

# Sort the DataFrame by date for easier analysis and visualization
original = resulting_df.sort_values(by='date')

# Output the resulting DataFrame
print(resulting_df.head())


/var/folders/bm/bm_nkxfx3w1303dr6gyz9cg00000gn/T/ipykernel_35322/457891423.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top500_last_day = data_last_day.groupby('MonthYear').apply(lambda x: x.nlargest(500, 'MktCap')).reset_index(drop=True)


   PERMNO       date TICKER  FACPR  FACSHR  BIDLO  ASKHI    PRC         VOL  \
0   10104 2016-02-01   ORCL    NaN     NaN  35.85  36.54  36.32  14143998.0   
1   10104 2016-02-02   ORCL    NaN     NaN  35.25  35.94  35.35  17404484.0   
2   10104 2016-02-03   ORCL    NaN     NaN  34.94  35.79  35.69  13855330.0   
3   10104 2016-02-04   ORCL    NaN     NaN  35.66  36.54  36.45  16925925.0   
4   10104 2016-02-05   ORCL    NaN     NaN  35.37  36.24  35.70  17312179.0   

        RET     SHROUT  CFACPR  CFACSHR  OPENPRC MonthYear       MktCap  \
0  0.000275  4201220.0     1.0      1.0    35.90   2016-02  152588310.4   
1 -0.026707  4201220.0     1.0      1.0    35.85   2016-02  148513127.0   
2  0.009618  4201220.0     1.0      1.0    35.56   2016-02  149941541.8   
3  0.021295  4201220.0     1.0      1.0    35.70   2016-02  153134469.0   
4 -0.020576  4201220.0     1.0      1.0    36.13   2016-02  149983554.0   

  NextMonth  
0   2016-02  
1   2016-02  
2   2016-02  
3   2016-02  
4   

In [41]:
resulting_df=original

In [42]:
resulting_df

,PERMNO,date,TICKER,FACPR,FACSHR,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,OPENPRC,MonthYear,MktCap,NextMonth
0,10104,2016-02-01,ORCL,NaN,NaN,35.85000,36.54000,36.32,14143998.0,0.000275,4201220.0,1.000000,1.0,35.90000,2016-02,1.525883e+08,2016-02
778598,88219,2016-02-01,IWD,NaN,NaN,91.60000,92.91000,92.43,3434694.0,-0.002805,264650.0,1.000000,1.0,92.10000,2016-02,2.446160e+07,2016-02
55866,12449,2016-02-01,FLT,NaN,NaN,120.36000,122.75000,121.76,514935.0,-0.008792,92161.0,1.000000,1.0,121.24000,2016-02,1.122152e+07,2016-02
923597,91152,2016-02-01,TDG,NaN,NaN,221.74500,230.00000,228.69,446212.0,0.017621,53592.0,1.000000,1.0,223.48000,2016-02,1.225595e+07,2016-02
946594,91937,2016-02-01,TMUS,NaN,NaN,39.46000,40.65000,40.59,3645991.0,0.010959,818391.0,1.003412,1.0,40.04000,2016-02,3.321849e+07,2016-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740130,86580,2023-12-29,NVDA,NaN,NaN,487.51001,499.97000,495.22,38725363.0,0.000000,2470000.0,1.000000,1.0,498.13000,2023-12,1.223193e+09,2023-12
738137,86457,2023-12-29,XLK,NaN,NaN,191.52000,193.55991,192.48,6095325.0,-0.003056,304106.0,1.000000,1.0,193.17000,2023-12,5.853432e+07,2023-12
181199,16087,2023-12-29,FTV,NaN,NaN,73.30000,73.99190,73.63,874248.0,0.000679,351434.0,1.000000,1.0,73.46000,2023-12,2.587609e+07,2023-12
183149,16309,2023-12-29,TTD,NaN,NaN,71.64000,73.48050,71.96,3744952.0,-0.019619,446261.0,1.000000,1.0,73.30000,2023-12,3.211294e+07,2023-12


In [43]:
# #Rename and form WQ columns
wq_alphas=resulting_df.rename(columns={
    'OPENPRC': 'S_DQ_OPEN', 
    'ASKHI': 'S_DQ_HIGH', 
    'BIDLO': 'S_DQ_LOW', 
    'PRC': 'S_DQ_CLOSE', 
    'VOL': 'S_DQ_VOLUME', 
    'RET': 'S_DQ_PCTCHANGE',
})
wq_alphas=wq_alphas[['date','TICKER','S_DQ_OPEN','S_DQ_HIGH','S_DQ_LOW','S_DQ_CLOSE','S_DQ_VOLUME','S_DQ_PCTCHANGE']]
wq_alphas

,date,TICKER,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_PCTCHANGE
0,2016-02-01,ORCL,35.90000,36.54000,35.85000,36.32,14143998.0,0.000275
778598,2016-02-01,IWD,92.10000,92.91000,91.60000,92.43,3434694.0,-0.002805
55866,2016-02-01,FLT,121.24000,122.75000,120.36000,121.76,514935.0,-0.008792
923597,2016-02-01,TDG,223.48000,230.00000,221.74500,228.69,446212.0,0.017621
946594,2016-02-01,TMUS,40.04000,40.65000,39.46000,40.59,3645991.0,0.010959
...,...,...,...,...,...,...,...,...
740130,2023-12-29,NVDA,498.13000,499.97000,487.51001,495.22,38725363.0,0.000000
738137,2023-12-29,XLK,193.17000,193.55991,191.52000,192.48,6095325.0,-0.003056
181199,2023-12-29,FTV,73.46000,73.99190,73.30000,73.63,874248.0,0.000679
183149,2023-12-29,TTD,73.30000,73.48050,71.64000,71.96,3744952.0,-0.019619


In [44]:
wq_alphas['S_DQ_VOLUME'] = wq_alphas['S_DQ_VOLUME'] 

# Calculate S_DQ_AMOUNT as the product of S_DQ_CLOSE and S_DQ_VOLUME
wq_alphas['S_DQ_AMOUNT'] = wq_alphas['S_DQ_CLOSE'] * wq_alphas['S_DQ_VOLUME']

# Calculate VWAP
wq_alphas['vwap'] = wq_alphas['S_DQ_AMOUNT'] / wq_alphas['S_DQ_VOLUME']

In [45]:
wq_alphas.to_csv('wq_alphas_modern.csv')

In [9]:
# from wq_alpha_generator import get_alpha
# def apply_get_alpha_to_each_stock(group_df):
#     return get_alpha(group_df)

# # Group by 'TICKER' and apply the function
# result_df = wq_alphas.groupby('TICKER').apply(apply_get_alpha_to_each_stock).reset_index(drop=True)


In [10]:
# df=pd.read_csv('../wq_alphas.csv')

In [11]:
# df.sort_values(by=['TICKER', 'date'], inplace=True)


In [12]:
# resulting_df=resulting_df.drop(['NUMTRD'],axis=1)

In [13]:
resulting_df['Previous Close'] = resulting_df['PRC'].shift(1)
resulting_df['High_Low'] = resulting_df['ASKHI'] - resulting_df['BIDLO']
resulting_df['High_PrevClose'] = abs(resulting_df['ASKHI'] - resulting_df['PRC'])
resulting_df['Low_PrevClose'] = abs(resulting_df['BIDLO'] - resulting_df['PRC'])

resulting_df['TR'] = resulting_df[['High_Low', 'High_PrevClose', 'Low_PrevClose']].max(axis=1)

# Calculate the ATR
resulting_df['ATR'] = resulting_df['TR'].rolling(window=14).mean()


In [14]:
resulting_df=resulting_df.drop(['Previous Close','High_Low','High_PrevClose','Low_PrevClose'],axis=1)

In [15]:
resulting_df

,PERMNO,date,TICKER,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,OPENPRC,NUMTRD,MonthYear,MktCap,NextMonth,TR,ATR
0,10078,2000-02-01,SUNW,76.50000,81.500,80.7500,15724295.0,0.027844,1561106.0,0.500000,0.50,78.87500,24586.0,2000-02,1.260593e+08,2000-02,5.00000,NaN
1382557,79879,2000-02-01,JDSU,194.68750,206.500,202.3125,10591961.0,-0.007968,248261.0,0.419471,0.25,204.76563,33412.0,2000-02,5.022630e+07,2000-02,11.81250,NaN
1763557,87343,2000-02-01,SCMR,300.87500,319.125,304.0000,607444.0,-0.047022,78696.0,0.300000,0.30,318.50000,1240.0,2000-02,2.392358e+07,2000-02,18.25000,NaN
1764291,87359,2000-02-01,ITRU,137.37500,153.500,147.6250,284516.0,0.022511,37751.0,2.000000,2.00,147.00000,592.0,2000-02,5.572991e+06,2000-02,16.12500,NaN
1764334,87368,2000-02-01,SNRA,71.93750,75.750,74.4375,635920.0,0.074910,113000.0,1.243927,1.00,72.93750,1162.0,2000-02,8.411438e+06,2000-02,3.81250,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948049,89626,2016-12-30,CME,114.90000,116.480,115.3500,898599.0,-0.002076,338240.0,1.000000,1.00,116.07000,9341.0,2016-12,3.901598e+07,2016-12,1.58000,0.818658
593892,30681,2016-12-30,OMC,84.40000,85.790,85.1100,1737326.0,-0.006653,235881.0,1.000000,1.00,85.64000,NaN,2016-12,2.007583e+07,2016-12,1.39000,0.892229
1944964,89617,2016-12-30,EQIX,353.48001,358.340,357.4100,501346.0,0.000896,71409.0,1.000000,1.00,358.14999,7133.0,2016-12,2.552229e+07,2016-12,4.85999,1.213300
604792,33099,2016-12-30,IMO,34.68000,35.360,34.7600,115389.0,-0.005152,847599.0,1.000000,1.00,34.82000,NaN,2016-12,2.946254e+07,2016-12,0.68000,1.209014


In [16]:
# Rate of Change (ROC) for PRC over 2 periods
resulting_df['ROC'] = resulting_df['PRC'].pct_change(periods=2)

# Moving Average Convergence Divergence (MACD)
# Fast EMA (12 periods) - Slow EMA (26 periods) for this example
resulting_df['EMA12'] = resulting_df['PRC'].ewm(span=12, adjust=False).mean()
resulting_df['EMA26'] = resulting_df['PRC'].ewm(span=26, adjust=False).mean()
resulting_df['MACD'] = resulting_df['EMA12'] - resulting_df['EMA26']

resulting_df['VWAP'] = (resulting_df['VOL'] * ((resulting_df['ASKHI'] + resulting_df['BIDLO'] + resulting_df['PRC']) / 3)).cumsum() / resulting_df['VOL'].cumsum()

resulting_df['SMA20'] = resulting_df['PRC'].rolling(window=20).mean()
resulting_df['STD20'] = resulting_df['PRC'].rolling(window=20).std()
resulting_df['UpperBand'] = resulting_df['SMA20'] + (2 * resulting_df['STD20'])
resulting_df['LowerBand'] = resulting_df['SMA20'] - (2 * resulting_df['STD20'])

resulting_df['OBV'] = (np.sign(resulting_df['RET']) * resulting_df['VOL']).cumsum()



In [17]:
resulting_df

,PERMNO,date,TICKER,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,CFACPR,...,ROC,EMA12,EMA26,MACD,VWAP,SMA20,STD20,UpperBand,LowerBand,OBV
0,10078,2000-02-01,SUNW,76.50000,81.500,80.7500,15724295.0,0.027844,1561106.0,0.500000,...,NaN,80.750000,80.750000,0.000000,79.583333,NaN,NaN,NaN,NaN,1.572430e+07
1382557,79879,2000-02-01,JDSU,194.68750,206.500,202.3125,10591961.0,-0.007968,248261.0,0.419471,...,NaN,99.451923,89.754630,9.697293,128.519091,NaN,NaN,NaN,NaN,5.132334e+06
1763557,87343,2000-02-01,SCMR,300.87500,319.125,304.0000,607444.0,-0.047022,78696.0,0.300000,...,2.764706,130.920858,105.624657,25.296201,132.568482,NaN,NaN,NaN,NaN,4.524890e+06
1764291,87359,2000-02-01,ITRU,137.37500,153.500,147.6250,284516.0,0.022511,37751.0,2.000000,...,-0.270312,133.490726,108.735794,24.754932,132.710678,NaN,NaN,NaN,NaN,4.809406e+06
1764334,87368,2000-02-01,SNRA,71.93750,75.750,74.4375,635920.0,0.074910,113000.0,1.243927,...,-0.755140,124.405614,106.195179,18.210435,131.370763,NaN,NaN,NaN,NaN,5.445326e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948049,89626,2016-12-30,CME,114.90000,116.480,115.3500,898599.0,-0.002076,338240.0,1.000000,...,1.667669,68.076154,69.185553,-1.109398,44.441106,62.617501,39.058785,140.735071,-15.500070,6.889350e+10
593892,30681,2016-12-30,OMC,84.40000,85.790,85.1100,1737326.0,-0.006653,235881.0,1.000000,...,1.276886,70.696746,70.365141,0.331605,44.441112,60.039501,35.448557,130.936615,-10.857614,6.889176e+10
1944964,89617,2016-12-30,EQIX,353.48001,358.340,357.4100,501346.0,0.000896,71409.0,1.000000,...,2.098483,114.806477,91.627723,23.178754,44.441124,74.934001,75.347415,225.628830,-75.760829,6.889226e+10
604792,33099,2016-12-30,IMO,34.68000,35.360,34.7600,115389.0,-0.005152,847599.0,1.000000,...,-0.591587,102.491635,87.415299,15.076335,44.441124,76.065501,74.520018,225.105536,-72.974535,6.889214e+10


In [18]:
filtered_df = resulting_df[~((resulting_df['ASKHI'] == 0) & (resulting_df['BIDLO'] == 0))]
filtered_df=filtered_df.drop(['BIDLO','ASKHI'],axis=1)
filtered_df

,PERMNO,date,TICKER,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,OPENPRC,...,ROC,EMA12,EMA26,MACD,VWAP,SMA20,STD20,UpperBand,LowerBand,OBV
0,10078,2000-02-01,SUNW,80.7500,15724295.0,0.027844,1561106.0,0.500000,0.50,78.87500,...,NaN,80.750000,80.750000,0.000000,79.583333,NaN,NaN,NaN,NaN,1.572430e+07
1382557,79879,2000-02-01,JDSU,202.3125,10591961.0,-0.007968,248261.0,0.419471,0.25,204.76563,...,NaN,99.451923,89.754630,9.697293,128.519091,NaN,NaN,NaN,NaN,5.132334e+06
1763557,87343,2000-02-01,SCMR,304.0000,607444.0,-0.047022,78696.0,0.300000,0.30,318.50000,...,2.764706,130.920858,105.624657,25.296201,132.568482,NaN,NaN,NaN,NaN,4.524890e+06
1764291,87359,2000-02-01,ITRU,147.6250,284516.0,0.022511,37751.0,2.000000,2.00,147.00000,...,-0.270312,133.490726,108.735794,24.754932,132.710678,NaN,NaN,NaN,NaN,4.809406e+06
1764334,87368,2000-02-01,SNRA,74.4375,635920.0,0.074910,113000.0,1.243927,1.00,72.93750,...,-0.755140,124.405614,106.195179,18.210435,131.370763,NaN,NaN,NaN,NaN,5.445326e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948049,89626,2016-12-30,CME,115.3500,898599.0,-0.002076,338240.0,1.000000,1.00,116.07000,...,1.667669,68.076154,69.185553,-1.109398,44.441106,62.617501,39.058785,140.735071,-15.500070,6.889350e+10
593892,30681,2016-12-30,OMC,85.1100,1737326.0,-0.006653,235881.0,1.000000,1.00,85.64000,...,1.276886,70.696746,70.365141,0.331605,44.441112,60.039501,35.448557,130.936615,-10.857614,6.889176e+10
1944964,89617,2016-12-30,EQIX,357.4100,501346.0,0.000896,71409.0,1.000000,1.00,358.14999,...,2.098483,114.806477,91.627723,23.178754,44.441124,74.934001,75.347415,225.628830,-75.760829,6.889226e+10
604792,33099,2016-12-30,IMO,34.7600,115389.0,-0.005152,847599.0,1.000000,1.00,34.82000,...,-0.591587,102.491635,87.415299,15.076335,44.441124,76.065501,74.520018,225.105536,-72.974535,6.889214e+10


In [20]:
filtered_df=filtered_df.drop(['NextMonth','MonthYear','CFACPR','CFACSHR','SHROUT'],axis=1)
filtered_df=filtered_df.dropna()
filtered_df

,PERMNO,date,TICKER,PRC,VOL,RET,OPENPRC,NUMTRD,MktCap,TR,...,ROC,EMA12,EMA26,MACD,VWAP,SMA20,STD20,UpperBand,LowerBand,OBV
1773396,87455,2000-02-01,NXTV,81.0000,272468.0,0.004651,82.00000,383.0,6.355827e+06,4.75000,...,-0.237647,71.843197,75.743367,-3.900170,98.807239,83.707813,67.105012,217.917837,-50.502212,2.672771e+06
1773629,87489,2000-02-01,FMKT,225.3750,246104.0,-0.015830,233.75000,1190.0,7.652608e+06,18.87500,...,1.782407,89.967614,83.664066,6.303548,98.459174,71.142188,47.461616,166.065420,-23.781045,3.146167e+06
1337539,78975,2000-02-01,INTU,64.8125,2260595.0,0.074611,62.18750,4299.0,1.210198e+07,5.62500,...,-0.712424,76.598854,77.695359,-1.096505,92.449659,64.129688,45.263529,154.656745,-26.397370,2.184363e+06
1330541,78840,2000-02-01,USAI,49.9375,1091399.0,0.006297,49.62500,1163.0,6.816668e+06,0.87500,...,1.937500,72.497107,75.639221,-3.142114,91.647891,63.573437,45.371435,154.316308,-27.169433,3.275762e+06
1383722,79882,2000-02-01,TVGIA,38.6250,818723.0,0.042159,37.46875,574.0,5.975906e+06,2.25000,...,0.181644,57.856975,67.296650,-9.439674,84.735179,60.992188,45.253548,151.499284,-29.514909,1.091558e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611774,34746,2016-12-30,FITB,26.9700,4901569.0,0.008979,26.83000,30848.0,2.024042e+07,0.45000,...,-0.546875,65.270840,77.364714,-12.093874,44.441119,80.623499,64.539997,209.703493,-48.456496,6.897687e+10
1951707,89641,2016-12-30,STX,38.1700,2897495.0,0.003945,38.18000,23246.0,1.126595e+07,0.67000,...,-0.734599,61.472778,67.568373,-6.095595,44.441097,66.072501,44.928334,155.929169,-23.784168,6.889983e+10
1948049,89626,2016-12-30,CME,115.3500,898599.0,-0.002076,116.07000,9341.0,3.901598e+07,1.58000,...,1.667669,68.076154,69.185553,-1.109398,44.441106,62.617501,39.058785,140.735071,-15.500070,6.889350e+10
1944964,89617,2016-12-30,EQIX,357.4100,501346.0,0.000896,358.14999,7133.0,2.552229e+07,4.85999,...,2.098483,114.806477,91.627723,23.178754,44.441124,74.934001,75.347415,225.628830,-75.760829,6.889226e+10


In [21]:
filtered_df=filtered_df.drop(['TR','EMA12','EMA26','OPENPRC'],axis=1)

In [22]:
filtered_df

,PERMNO,date,TICKER,PRC,VOL,RET,NUMTRD,MktCap,ATR,ROC,MACD,VWAP,SMA20,STD20,UpperBand,LowerBand,OBV
1773396,87455,2000-02-01,NXTV,81.0000,272468.0,0.004651,383.0,6.355827e+06,2.580357,-0.237647,-3.900170,98.807239,83.707813,67.105012,217.917837,-50.502212,2.672771e+06
1773629,87489,2000-02-01,FMKT,225.3750,246104.0,-0.015830,1190.0,7.652608e+06,3.589286,1.782407,6.303548,98.459174,71.142188,47.461616,166.065420,-23.781045,3.146167e+06
1337539,78975,2000-02-01,INTU,64.8125,2260595.0,0.074611,4299.0,1.210198e+07,3.723214,-0.712424,-1.096505,92.449659,64.129688,45.263529,154.656745,-26.397370,2.184363e+06
1330541,78840,2000-02-01,USAI,49.9375,1091399.0,0.006297,1163.0,6.816668e+06,3.714286,1.937500,-3.142114,91.647891,63.573437,45.371435,154.316308,-27.169433,3.275762e+06
1383722,79882,2000-02-01,TVGIA,38.6250,818723.0,0.042159,574.0,5.975906e+06,3.950893,0.181644,-9.439674,84.735179,60.992188,45.253548,151.499284,-29.514909,1.091558e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611774,34746,2016-12-30,FITB,26.9700,4901569.0,0.008979,30848.0,2.024042e+07,0.878287,-0.546875,-12.093874,44.441119,80.623499,64.539997,209.703493,-48.456496,6.897687e+10
1951707,89641,2016-12-30,STX,38.1700,2897495.0,0.003945,23246.0,1.126595e+07,0.706429,-0.734599,-6.095595,44.441097,66.072501,44.928334,155.929169,-23.784168,6.889983e+10
1948049,89626,2016-12-30,CME,115.3500,898599.0,-0.002076,9341.0,3.901598e+07,0.818658,1.667669,-1.109398,44.441106,62.617501,39.058785,140.735071,-15.500070,6.889350e+10
1944964,89617,2016-12-30,EQIX,357.4100,501346.0,0.000896,7133.0,2.552229e+07,1.213300,2.098483,23.178754,44.441124,74.934001,75.347415,225.628830,-75.760829,6.889226e+10


In [23]:
filtered_df['Delta'] = filtered_df.groupby('TICKER')['PRC'].diff()

# Separate gains and losses
filtered_df['Gain'] = np.where(filtered_df['Delta'] > 0, filtered_df['Delta'], 0)
filtered_df['Loss'] = np.where(filtered_df['Delta'] < 0, -filtered_df['Delta'], 0)

# Calculate exponential moving average of gains and losses
filtered_df['Avg_Gain'] = filtered_df.groupby('TICKER')['Gain'].transform(lambda x: x.ewm(span=14).mean())
filtered_df['Avg_Loss'] = filtered_df.groupby('TICKER')['Loss'].transform(lambda x: x.ewm(span=14).mean())

# Calculate RS and RSI
filtered_df['RS'] = filtered_df['Avg_Gain'] / filtered_df['Avg_Loss']
filtered_df['RSI'] = 100 - (100 / (1 + filtered_df['RS']))

In [24]:
filtered_df['PVT'] = filtered_df.groupby('TICKER').apply(lambda x: (x['VOL'] * x['RET']).cumsum()).reset_index(level=0, drop=True)

/var/folders/bm/bm_nkxfx3w1303dr6gyz9cg00000gn/T/ipykernel_35322/3685880761.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df['PVT'] = filtered_df.groupby('TICKER').apply(lambda x: (x['VOL'] * x['RET']).cumsum()).reset_index(level=0, drop=True)


In [25]:
filtered_df

,PERMNO,date,TICKER,PRC,VOL,RET,NUMTRD,MktCap,ATR,ROC,...,LowerBand,OBV,Delta,Gain,Loss,Avg_Gain,Avg_Loss,RS,RSI,PVT
1773396,87455,2000-02-01,NXTV,81.0000,272468.0,0.004651,383.0,6.355827e+06,2.580357,-0.237647,...,-50.502212,2.672771e+06,NaN,0.00,0.00000,0.000000,0.000000,NaN,NaN,1.267249e+03
1773629,87489,2000-02-01,FMKT,225.3750,246104.0,-0.015830,1190.0,7.652608e+06,3.589286,1.782407,...,-23.781045,3.146167e+06,NaN,0.00,0.00000,0.000000,0.000000,NaN,NaN,-3.895826e+03
1337539,78975,2000-02-01,INTU,64.8125,2260595.0,0.074611,4299.0,1.210198e+07,3.723214,-0.712424,...,-26.397370,2.184363e+06,NaN,0.00,0.00000,0.000000,0.000000,NaN,NaN,1.686653e+05
1330541,78840,2000-02-01,USAI,49.9375,1091399.0,0.006297,1163.0,6.816668e+06,3.714286,1.937500,...,-27.169433,3.275762e+06,NaN,0.00,0.00000,0.000000,0.000000,NaN,NaN,6.872540e+03
1383722,79882,2000-02-01,TVGIA,38.6250,818723.0,0.042159,574.0,5.975906e+06,3.950893,0.181644,...,-29.514909,1.091558e+07,NaN,0.00,0.00000,0.000000,0.000000,NaN,NaN,3.451654e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611774,34746,2016-12-30,FITB,26.9700,4901569.0,0.008979,30848.0,2.024042e+07,0.878287,-0.546875,...,-48.456496,6.897687e+10,0.24000,0.24,0.00000,0.138870,0.156443,0.887671,47.024680,-2.104603e+07
1951707,89641,2016-12-30,STX,38.1700,2897495.0,0.003945,23246.0,1.126595e+07,0.706429,-0.734599,...,-23.784168,6.889983e+10,0.15000,0.15,0.00000,0.188405,0.270779,0.695787,41.030324,9.371641e+06
1948049,89626,2016-12-30,CME,115.3500,898599.0,-0.002076,9341.0,3.901598e+07,0.818658,1.667669,...,-15.500070,6.889350e+10,-0.24000,0.00,0.24000,0.195559,0.638822,0.306124,23.437603,4.134286e+06
1944964,89617,2016-12-30,EQIX,357.4100,501346.0,0.000896,7133.0,2.552229e+07,1.213300,2.098483,...,-75.760829,6.889226e+10,0.32000,0.32,0.00000,1.820769,1.111172,1.638602,62.101142,8.233424e+05


In [26]:
filtered_df=filtered_df.dropna()

In [28]:
filtered_df=filtered_df.drop(['RS','Gain','Loss','Avg_Gain','Avg_Loss','Delta'],axis=1)

In [30]:
filtered_df['Momentum_TradeCount'] = filtered_df['NUMTRD'] - filtered_df['NUMTRD'].shift(10)
filtered_df['EMA_TradeCount'] = filtered_df['NUMTRD'].ewm(span=20).mean()



In [31]:
filtered_df

,PERMNO,date,TICKER,PRC,VOL,RET,NUMTRD,MktCap,ATR,ROC,...,VWAP,SMA20,STD20,UpperBand,LowerBand,OBV,RSI,PVT,Momentum_TradeCount,EMA_TradeCount
894566,59328,2000-02-02,INTC,100.0625,26376510.0,-0.013555,32635.0,3.343088e+08,2.128349,1.340643,...,72.066923,66.643750,48.617296,163.878342,-30.590842,5.080492e+08,0.000000,2.551634e+05,NaN,32635.000000
2016077,90609,2000-02-02,NOVL,35.3125,3973305.0,-0.008772,5698.0,1.153073e+07,1.606027,0.162551,...,71.908562,66.415625,48.514594,163.444813,-30.613563,5.039555e+08,0.000000,3.402526e+05,NaN,18493.075000
1259509,77178,2000-02-02,QCOM,142.0625,29150844.0,0.044097,70788.0,9.351633e+07,2.424107,2.174581,...,73.394075,62.456250,36.224549,134.905349,-9.992849,5.390182e+08,100.000000,3.812824e+06,NaN,37695.457952
1143555,75257,2000-02-02,TLAB,56.6250,6656445.0,0.011161,8082.0,2.277888e+07,2.531250,0.592267,...,73.316368,61.071875,35.872669,132.817213,-10.673463,5.456746e+08,100.000000,3.188838e+05,NaN,29146.510505
1400173,80266,2000-02-02,QLGC,165.0000,857817.0,0.050955,2591.0,5.969040e+06,3.276786,0.161461,...,73.370746,67.184375,42.406697,151.997768,-17.629018,5.465324e+08,100.000000,6.377849e+04,NaN,22722.958496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611774,34746,2016-12-30,FITB,26.9700,4901569.0,0.008979,30848.0,2.024042e+07,0.878287,-0.546875,...,44.441119,80.623499,64.539997,209.703493,-48.456496,6.897687e+10,47.024680,-2.104603e+07,22636.0,24210.807396
1951707,89641,2016-12-30,STX,38.1700,2897495.0,0.003945,23246.0,1.126595e+07,0.706429,-0.734599,...,44.441097,66.072501,44.928334,155.929169,-23.784168,6.889983e+10,41.030324,9.371641e+06,-51756.0,24118.920978
1948049,89626,2016-12-30,CME,115.3500,898599.0,-0.002076,9341.0,3.901598e+07,0.818658,1.667669,...,44.441106,62.617501,39.058785,140.735071,-15.500070,6.889350e+10,23.437603,4.134286e+06,5271.0,22711.499932
1944964,89617,2016-12-30,EQIX,357.4100,501346.0,0.000896,7133.0,2.552229e+07,1.213300,2.098483,...,44.441124,74.934001,75.347415,225.628830,-75.760829,6.889226e+10,62.101142,8.233424e+05,-9653.0,21227.833272


In [32]:
filtered_df.to_parquet('../data/technical_stock_data.parquet')

In [36]:
import statsmodels.api as sm

# Choose the independent variable, for example, 'ATR'
X = filtered_df['RSI']
# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Choose the dependent variable
Y = filtered_df['RET']

# Conduct the regression
model = sm.OLS(Y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                 5.555e+04
Date:                Tue, 12 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:49:21   Log-Likelihood:             7.8317e+05
No. Observations:              373874   AIC:                        -1.566e+06
Df Residuals:                  373872   BIC:                        -1.566e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0355      0.000   -223.123      0.0

# Word2Vec Embeddings

In [12]:
from gensim.models import Word2Vec

# Load the trained Word2Vec model
model = Word2Vec.load("../adjusted_stock_embeddings.model")

In [14]:
import numpy as np

# Example DataFrame
df = pd.read_parquet('../stock_data_w_alphas.parquet')

# Initialize a list to hold the embeddings
embeddings = []

# Iterate over the DataFrame
for ticker in df['TICKER']:
    if ticker in model.wv:  # Check if the model has the ticker
        embedding = model.wv[ticker]
    else:
        embedding = np.zeros(model.vector_size)  # Use a zero vector if the ticker is not in the model
    embeddings.append(embedding)

# Convert the list of embeddings into a numpy array
embeddings = np.array(embeddings)

# Add the embeddings as new features to the DataFrame. This results in a DataFrame where each row represents a day/stock and includes the embedding vector as features.
# Note: This will expand the embeddings into separate columns for each dimension of the embedding.
for i in range(embeddings.shape[1]):  # For each dimension in the embedding
    df[f'emb_{i}'] = embeddings[:, i]

In [15]:
df

,date,TICKER,S_DQ_CLOSE,RET,alpha001,alpha002,alpha003,alpha004,alpha005,alpha006,...,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49
124471,2001-01-26,SUNW,31.18750,0.000000,0.433086,0.386525,-0.237328,-5.0,-0.444184,-0.304482,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
124913,2001-01-29,SUNW,33.56250,0.076152,0.856081,-0.503691,-0.304077,-5.0,-0.444717,-0.358804,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
125879,2001-01-30,SUNW,32.06250,-0.044693,0.618694,-0.343073,-0.066061,-7.0,-0.447516,-0.218272,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
126189,2001-01-31,SUNW,30.56250,-0.046784,0.433086,-0.361457,-0.063250,-4.0,-0.446583,-0.238734,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
135113,2001-02-27,SUNW,19.81250,-0.045181,0.252788,-0.433718,0.673484,-4.0,-0.433786,0.623345,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133113,2016-12-22,NTES,217.10001,-0.026981,0.595618,-0.041233,0.266548,-6.0,-0.386488,0.267122,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2133490,2016-12-23,NTES,223.67000,0.030263,0.848606,0.304483,-0.068472,-5.0,-0.360475,-0.051078,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134129,2016-12-27,NTES,223.99001,0.001431,0.848606,0.142212,0.395631,-9.0,-0.397637,0.454995,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134924,2016-12-28,NTES,223.92000,-0.000313,0.595618,0.095192,0.547896,-8.0,-0.405070,0.586525,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621


In [16]:
df = df.drop(columns=df.filter(like='alpha').columns)

In [17]:
df

,date,TICKER,S_DQ_CLOSE,RET,1D_forward_return,emb_0,emb_1,emb_2,emb_3,emb_4,...,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49
124471,2001-01-26,SUNW,31.18750,0.000000,-0.070764,1.525310,-0.612527,2.872295,-1.253325,0.825628,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
124913,2001-01-29,SUNW,33.56250,0.076152,0.046784,1.525310,-0.612527,2.872295,-1.253325,0.825628,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
125879,2001-01-30,SUNW,32.06250,-0.044693,0.049080,1.525310,-0.612527,2.872295,-1.253325,0.825628,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
126189,2001-01-31,SUNW,30.56250,-0.046784,0.542587,1.525310,-0.612527,2.872295,-1.253325,0.825628,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
135113,2001-02-27,SUNW,19.81250,-0.045181,-0.003145,1.525310,-0.612527,2.872295,-1.253325,0.825628,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133113,2016-12-22,NTES,217.10001,-0.026981,-0.029374,-0.048115,-0.102475,0.028629,0.194354,-0.124991,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2133490,2016-12-23,NTES,223.67000,0.030263,-0.001429,-0.048115,-0.102475,0.028629,0.194354,-0.124991,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134129,2016-12-27,NTES,223.99001,0.001431,0.000313,-0.048115,-0.102475,0.028629,0.194354,-0.124991,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134924,2016-12-28,NTES,223.92000,-0.000313,0.004126,-0.048115,-0.102475,0.028629,0.194354,-0.124991,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621


In [19]:
df=df.drop(['1D_forward_return','S_DQ_CLOSE'],axis=1)

In [22]:
df

,date,TICKER,RET,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49
124471,2001-01-26,SUNW,0.000000,1.525310,-0.612527,2.872295,-1.253325,0.825628,1.064456,-2.778073,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
124913,2001-01-29,SUNW,0.076152,1.525310,-0.612527,2.872295,-1.253325,0.825628,1.064456,-2.778073,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
125879,2001-01-30,SUNW,-0.044693,1.525310,-0.612527,2.872295,-1.253325,0.825628,1.064456,-2.778073,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
126189,2001-01-31,SUNW,-0.046784,1.525310,-0.612527,2.872295,-1.253325,0.825628,1.064456,-2.778073,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
135113,2001-02-27,SUNW,-0.045181,1.525310,-0.612527,2.872295,-1.253325,0.825628,1.064456,-2.778073,...,-1.324405,1.840051,-1.305009,-1.272032,0.232388,-3.453357,-1.717485,-0.249291,0.176502,0.624516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133113,2016-12-22,NTES,-0.026981,-0.048115,-0.102475,0.028629,0.194354,-0.124991,0.022221,0.016089,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2133490,2016-12-23,NTES,0.030263,-0.048115,-0.102475,0.028629,0.194354,-0.124991,0.022221,0.016089,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134129,2016-12-27,NTES,0.001431,-0.048115,-0.102475,0.028629,0.194354,-0.124991,0.022221,0.016089,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621
2134924,2016-12-28,NTES,-0.000313,-0.048115,-0.102475,0.028629,0.194354,-0.124991,0.022221,0.016089,...,0.028340,-0.018812,-0.082137,-0.054498,0.070861,0.095508,-0.119153,0.152850,0.074185,0.017621


In [ ]:
df.drop('')

In [21]:
df.to_parquet('stock_data_w_embeddings.parquet')